# Exercises

## Exercise 1
Write yet another function that solves for element n of the Fibonacci sequence,
using a technique of your own design. Write unit tests that evaluate its correctness
and performance relative to the other versions in this chapter.

### Solution
The Fibonacci numbers have a closed-form solution (https://en.wikipedia.org/wiki/Fibonacci_number#Closed-form_expression):

$F_n = \left \lfloor \frac{\phi^n}{\sqrt{5}}+\frac{1}{2} \right \rfloor$, with

$\phi = \frac{1+\sqrt{5}}{2}$

In [125]:
import math
def fib_closed(n: int) -> int:
    phi = (1 + math.sqrt(5))/2
    return math.floor((phi**n)/math.sqrt(5)+(1/2))

Unit tests

In [126]:
from fib5 import fib5 
from fib4 import fib4
import timeit
for i in [0,1,5,25, 50,75, 100]:
    print(f"fib5({i}) = {fib5(i)}, fib_closed({i})={fib_closed(i)}")

fib5(0) = 0, fib_closed(0)=0
fib5(1) = 1, fib_closed(1)=1
fib5(5) = 5, fib_closed(5)=5
fib5(25) = 75025, fib_closed(25)=75025
fib5(50) = 12586269025, fib_closed(50)=12586269025
fib5(75) = 2111485077978050, fib_closed(75)=2111485077978055
fib5(100) = 354224848179261915075, fib_closed(100)=354224848179263111168


Seems the closed form solution suffers from rounding errors when calculating very big fibonacci numbers.

In [117]:
from timeit import default_timer as timer

In [121]:
from typing import Callable
def fib_perf(fib: Callable[[int],int], n:int = 50, repeats:int = 1000000)->float:
    start = timer()
    for i in range(repeats):
        fib(n)
    end = timer()
    return(end - start)

In [122]:
for n in [5, 10, 25, 50]:
    print(f"fib_closed speedup for fib({n}): {fib_perf(fib5, n)/fib_perf(fib_closed, n)}")

fib_closed speedup for fib(5): 0.5656257381690774
fib_closed speedup for fib(10): 0.7970060094147272
fib_closed speedup for fib(25): 1.9666871906484276
fib_closed speedup for fib(50): 3.391618790332616
